In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, GlobalMaxPooling2D, Activation, GlobalAveragePooling2D
from tensorflow.keras import Model, Sequential

import tensorboard

import json

In [2]:
datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=60, shear_range=20)

In [3]:
PATH = "./dataset/"
train_gen = datagen.flow_from_directory(PATH, target_size=(225,225), class_mode='binary', classes=['normal', 'potholes'])
validation_gen = datagen.flow_from_directory(PATH, target_size=(225,225), class_mode='binary', classes=['normal', 'potholes'], subset='validation')

Found 681 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [43]:
resnet_model = keras.applications.ResNet50V2(weights='imagenet', input_shape=(225, 225, 3), include_top=False)

for layer in resnet_model.layers:
    layer.trainable=False

model = Sequential([
#     Conv2D(64, kernel_size=(2, 2), input_shape=(225, 225, 3), padding='same'),
#     Activation('relu'),
#     BatchNormalization(),
#     MaxPool2D(pool_size=(2,2),  strides=(2,2)),
    
#     Conv2D(128, kernel_size=(2, 2), padding='same'),
#     Activation('relu'),
#     BatchNormalization(),
#     MaxPool2D(pool_size=(2,2),  strides=(2,2)),
    
#     Conv2D(256, kernel_size=(2, 2), padding='same'),
#     Activation('relu'),
#     BatchNormalization(),
#     MaxPool2D(pool_size=(2,2),  strides=(2,2), padding='same'),
    
#     Conv2D(256, kernel_size=(2, 2)),
#     Activation('relu'),
#     BatchNormalization(),
#     MaxPool2D(pool_size=(2,2),  strides=(2,2), padding='same'),
    resnet_model,
    Flatten(),
    Dense(256),
    Dropout(0.5),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid')
    
])

In [37]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten_2 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               12845312  
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_619 (Activation)  (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
_________________________________________________________________
activation_620 (Activation)  (None, 1)                

In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
# Save Model Sturcture
model_stucture_save_path = "./Pothole_Model_Structure.json"
model_json = model.to_json()
with open(model_stucture_save_path, 'w') as json_file:
    json_file.write(model_json)

In [40]:
# Callbacks
model_save_path = './Pothole.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath=model_save_path, mode='max', monitor='accuracy', save_best_only=True, save_weights_only=True, verbose=1)

tensorboard_checkpoint = keras.callbacks.TensorBoard(log_dir="./logs/", write_graph=True, write_images=True, update_freq='batch')

In [41]:
!rm -rf ./logs/*

In [42]:
# %reload_ext tensorboard
# %load_ext tensorboard
# %tensorboard --logdir=./logs/
history = model.fit(train_gen,
                    validation_data=validation_gen,
                    epochs=20,
                    steps_per_epoch=int(580/32),
                    validation_steps=int(101/32),
                    callbacks=[checkpoint, tensorboard_checkpoint],
                    verbose=1
                   )

Epoch 1/20
18/18 [==============================] - ETA: 0s - loss: 6.6682 - accuracy: 0.7776
Epoch 00001: accuracy improved from -inf to 0.77758, saving model to ./Pothole.h5
18/18 [==============================] - 18s 975ms/step - loss: 6.6682 - accuracy: 0.7776
Epoch 2/20
18/18 [==============================] - ETA: 0s - loss: 1.8178 - accuracy: 0.9097
Epoch 00002: accuracy improved from 0.77758 to 0.90972, saving model to ./Pothole.h5
18/18 [==============================] - 15s 851ms/step - loss: 1.8178 - accuracy: 0.9097
Epoch 3/20
 2/18 [==>...........................] - ETA: 10s - loss: 0.1800 - accuracy: 0.9688

KeyboardInterrupt: 

In [ ]:
?keras

In [24]:
images = next(train_gen)

In [27]:
len(images)

2